In [50]:
import multiprocessing
import os
from langchain_community.chat_models import ChatLlamaCpp

In [51]:

# Path to your model weights
model_dir = "/Users/vinaytanwer/Desktop/Projects/Chatbots/LLM_checkpoints/NousResearch"
model_name = "Hermes-2-Pro-Llama-3-8B-Q8_0.gguf"
# model_name = "Hermes-2-Theta-Llama-3-70B-Q5_K_M.gguf"
local_model = os.path.join(model_dir, model_name)

In [52]:
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [218]:

llm = ChatLlamaCpp(
    temperature=0,
    model_path=local_model,
    n_ctx=10000,
    n_gpu_layers=-1,
    n_batch=300,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    # max_tokens=512,
    # n_threads=multiprocessing.cpu_count() - 1,
    # callback_manager=callback_manager,
    # repeat_penalty=0.5,
    # top_p=0.5,
    f16_kv=True,
    verbose=False,
)


### Invocation

In [55]:
response = llm.invoke("Describe a person in JSON format:")
# print(response, '\n')
# print(response.content)

{
  "name": {
    "first": "John",
    "last": "Doe"
  },
  "age": 35,
  "gender": "male",
  "address": {
    "street": "123 Main Street",
    "city": "New York",
    "state": "NY",
    "zip": "10001"
  },
  "phone_number": "+1 (234) 567-8901",
  "email": "john.doe@example.com"
}

In [56]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

ai_msg = llm.invoke(messages)
# ai_msg

J'aime programmer.

In [57]:
ai_msg.content

"J'aime programmer."

### Chaining

In [58]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
response = chain.invoke(
    {
        "input_language": "English",
        "output_language": "Hindi",
        "input": "I love programming.",
    }
)

मुझे प्रोग्रामिंग पसंद है।

In [59]:
print(response.content)

मुझे प्रोग्रामिंग पसंद है।


### Tool Calling

In [213]:
from langchain.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field


class WeatherInput(BaseModel):
    location: str = Field(description="The city and state, e.g. San Francisco, CA")
    unit: str = Field(enum=["celsius", "fahrenheit"])


@tool("get_current_weather", args_schema=WeatherInput)
def get_weather(location: str, unit: str):
    """Get the current weather in a given location"""
    return f"Now the weather in {location} is 22 {unit}"


llm_with_tools = llm.bind_tools(
    tools=[get_weather],   
    tool_choice={"type": "function", "function": {"name": "get_current_weather"}},
)



In [214]:

# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             """You are a helpful assistant.
#             Apart from your own knowledge base, following tools are binded to you using bind_tool method: 
#             1) get_current_weather - Get the current weather in a given location

#             Delegate the task to the tool regarding weather information. Do not make any wild guesses.
#             Say "I don't know" if neither you nor the tools can answer the user query
#             """
#         ),
#         ("human", "{input}"),
#     ]
# )


In [215]:
ai_msg = llm_with_tools.invoke(
    "what is the weather in New Delhi in celsius "
)

In [216]:
ai_msg.tool_calls

[{'name': 'get_current_weather',
  'args': {'location': 'New Delhi', 'unit': 'celsius'},
  'id': 'call__0_get_current_weather_cmpl-04a97cf4-2517-415c-b72b-a5037f4b6087'}]

In [217]:
ai_msg

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{ "location": "New Delhi", "unit": "celsius"}'}, 'tool_calls': [{'id': 'call__0_get_current_weather_cmpl-04a97cf4-2517-415c-b72b-a5037f4b6087', 'type': 'function', 'function': {'name': 'get_current_weather', 'arguments': '{ "location": "New Delhi", "unit": "celsius"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 20, 'completion_tokens': 15, 'total_tokens': 35}, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-193da2a5-8b4e-47c2-a217-2a5db65b1036-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'New Delhi', 'unit': 'celsius'}, 'id': 'call__0_get_current_weather_cmpl-04a97cf4-2517-415c-b72b-a5037f4b6087'}])

In [64]:
class MagicFunctionInput(BaseModel):
    magic_function_input: int = Field(description="The input value for magic function")


@tool("get_magic_function", args_schema=MagicFunctionInput)
def magic_function(magic_function_input: int):
    """Get the value of magic function for an input."""
    return magic_function_input + 2


llm_with_tools = llm.bind_tools(
    tools=[magic_function],
    tool_choice={"type": "function", "function": {"name": "get_magic_function"}},
)

ai_msg = llm_with_tools.invoke(
    "What is magic function of 3?",
)

ai_msg.tool_calls

[{'name': 'get_magic_function',
  'args': {'magic_function_input': 3},
  'id': 'call__0_get_magic_function_cmpl-4597a1db-3ede-4916-9ced-b015cc18e6cc'}]

### Structured output

In [65]:
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.utils.function_calling import convert_to_openai_tool


class Joke(BaseModel):
    """A setup to a joke and the punchline."""

    setup: str
    punchline: str


dict_schema = convert_to_openai_tool(Joke)
structured_llm = llm.with_structured_output(dict_schema)
result = structured_llm.invoke("Tell me a joke about batman")
result

{'setup': "Why don't Batman and Robin ever go on vacation together?",
 'punchline': "Because they'd be gone for R&R!"}

### Streaming

In [69]:
for chunk in llm.stream("Tell me a joke about Justice League characters. Keep the response concise"):
    # print(chunk.content, flush=True, end="")
    pass

Why did Batman refuse to join the Justice League? Because he couldn't find any Robin in it.

### Binding Multiple tools

##### This doesn't work

In [83]:
# @tool("get_magic_function", args_schema=MagicFunctionInput)
def fn_magic_function(magic_function_input: int):
    """Get the value of magic function for an input."""
    return magic_function_input + 2

# @tool("get_current_weather", args_schema=WeatherInput)
def fn_get_weather(location: str, unit: str):
    """Get the current weather in a given location"""
    return f"Now the weather in {location} is 22 {unit}"

llm_with_tools = llm.bind_tools(
    tools=[fn_magic_function, fn_get_weather],
    # tool_choice= {"type": "function", "function": {"name": "fn_get_weather"}}
    # tool_choice= {"type": "auto"}
)

In [79]:
ai_msg = llm_with_tools.invoke(
    "what is the current weather in HCMC in celsius",
)

ai_msg

To provide you with an accurate and up-to-date answer, I would need to access real-time data from a reliable source. Please allow me to check the current temperature in Ho Chi Minh City (HCMC) for you. 

Please wait for a moment while I gather this information for you.

AIMessage(content='To provide you with an accurate and up-to-date answer, I would need to access real-time data from a reliable source. Please allow me to check the current temperature in Ho Chi Minh City (HCMC) for you. \n\nPlease wait for a moment while I gather this information for you.', response_metadata={'finish_reason': 'stop'}, id='run-a925c698-58c0-4422-8956-0940c6717e9e-0')

In [80]:
ai_msg.tool_calls

[]

##### Workaround

In [207]:
class WeatherInput(BaseModel):
    location: str = Field(description="The city and state, e.g. San Francisco, CA")
    unit: str = Field(enum=["celsius", "fahrenheit"])

class MagicFunctionInput(BaseModel):
    magic_function_input: int = Field(description="The input value for magic function")

@tool("get_current_weather", args_schema=WeatherInput)
# @tool
def fn_get_weather(location: str, unit: str):
    """Get the current weather in a given location"""
    return f"Now the weather in {location} is 22 {unit}"

@tool("get_magic_function", args_schema=MagicFunctionInput)
def fn_magic_function(magic_function_input: int):
    """Get the value of magic function for an input."""
    return magic_function_input + 2

In [206]:
# fn_get_weather.__dict__
# fn_get_weather.args_schema.schema()

In [342]:
def get_function_signature(tools):
    signature_list = []
    for tool in tools:
        signature = {}
        signature["name"] = tool.name
        signature["description"] = tool.description
        signature["parameters"] = tool.args_schema.schema()
        signature_list.append(signature)

    return signature_list

tools = [fn_get_weather, fn_magic_function]
tools_signature = get_function_signature(tools)

print(f"{tools_signature}")


[{'name': 'get_current_weather', 'description': 'Get the current weather in a given location', 'parameters': {'title': 'WeatherInput', 'type': 'object', 'properties': {'location': {'title': 'Location', 'description': 'The city and state, e.g. San Francisco, CA', 'type': 'string'}, 'unit': {'title': 'Unit', 'enum': ['celsius', 'fahrenheit'], 'type': 'string'}}, 'required': ['location', 'unit']}}, {'name': 'get_magic_function', 'description': 'Get the value of magic function for an input.', 'parameters': {'title': 'MagicFunctionInput', 'type': 'object', 'properties': {'magic_function_input': {'title': 'Magic Function Input', 'description': 'The input value for magic function', 'type': 'integer'}}, 'required': ['magic_function_input']}}]


In [343]:
# display(tools_signature[0])

In [248]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import XMLOutputParser
from langchain_core.output_parsers.json import JsonOutputParser
import json

In [324]:
# prompt = PromptTemplate(
# template = """
# <|im_start|>system
# You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
# You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. 
# Here are the available tools: 
# <tools>
# {signatures}
# </tools>
# For each function call return a preamble statement followed by json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
# <tool_call>{{'arguments': <args-dict>, 'name: <function-name>}}</tool_call>
# <|im_end|>
# <|im_start|>user
# {query}
# <|im_end|>
# <im_start|>assistant
# """,
# input_variables=['signatures', 'query'])

In [332]:
prompt = PromptTemplate(
template = """
<|im_start|>system
You are a helpful assistant. You are also provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query if you are unsure about the answer. Don't make assumptions about what values to plug into functions. 
Here are the available tools: 
<tools>
{signatures}
</tools>
If you do not require function call to answer user query, respond in string format.
In case of a function call, do not include a preamble statement, only return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{{'arguments': <args-dict>, 'name: <function-name>}}</tool_call>
<|im_end|>
<|im_start|>user
{query}
<|im_end|>
""",
input_variables=['signatures', 'query'])

In [333]:
# print(prompt.invoke({'signatures':f"{tools_signature}", "query":"What is the current weather in New Delhi in celsius"}).text)

In [334]:
chain = prompt | llm.bind_tools(tools)
response = chain.invoke(
    {'signatures':f"{tools_signature}",
     "query":"What is the current weather in New Delhi in celsius",
    #  "query":"What is the magic function of 10",
    #  "query":"Tell me a joke above batman"
    }
)

response

AIMessage(content="<tool_call>{'arguments': {'location': 'New Delhi', 'unit': 'celsius'}, 'name': 'fn_get_weather'}</tool_call>", response_metadata={'finish_reason': 'stop'}, id='run-0a16b8c4-2cce-469f-849a-5456792eecd1-0')

In [357]:
from xml.etree import ElementTree as et
# import json
import ast

tool_call = et.fromstring(response.content)
tool_call = ast.literal_eval(tool_call.text)
tool_call

{'arguments': {'location': 'New Delhi', 'unit': 'celsius'},
 'name': 'fn_get_weather'}

In [361]:
tool_response = fn_get_weather.invoke(tool_call['arguments'])
tool_response

'Now the weather in New Delhi is 22 celsius'

In [370]:
prompt = PromptTemplate(
template = """
<|im_start|>system
You are a helpful assistant. You are also provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query if you are unsure about the answer. Don't make assumptions about what values to plug into functions.
Wait for the tool to respond to return the final answer in case of a tool call. Tool response would be provided in <tool_response></tool_response> XML tags.
Here are the available tools: 
<tools>
{signatures}
</tools>
If you do not require function call to answer user query, respond in string format.
In case of a function call, do not include a preamble statement, only return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>{{"arguments": <args-dict>, "name": <function-name>}}</tool_call>
<|im_end|>
<|im_start|>user
{query}
<|im_end|>
<|im_start|>assistant
<tool_call>{{'arguments': {{'location': 'New Delhi', 'unit': 'celsius'}}, 'name': 'fn_get_weather'}}</tool_call>
<|im_end|>
<|im_start|>tool
<tool_response>{{'content': 'Now the weather in New Delhi is 22 celsius', 'name': 'fn_get_weather'}}</tool_response>
<|im_end|>
""",
input_variables=['signatures', 'query'])

chain = prompt | llm.bind_tools(tools)
response = chain.invoke(
    {'signatures':f"{tools_signature}",
     "query":"What is the current weather in New Delhi in celsius",
    #  "query":"What is the magic function of 10",
    #  "query":"Tell me a joke above batman"
    }
)

response

AIMessage(content='The current weather in New Delhi is 22 degrees Celsius.', response_metadata={'finish_reason': 'stop'}, id='run-d628d8fc-2fad-4300-9b2c-5978bfb21833-0')

### With Chat Messages

In [381]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
import uuid

In [397]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """You are a helpful assistant. You are also provided with function signatures within <tools></tools> XML tags.
        You may call one or more functions to assist with the user query if you are unsure about the answer. Don't make assumptions about what values to plug into functions.
        Wait for the tool to respond to return the final answer in case of a tool call. Tool response would be provided in <tool_response></tool_response> XML tags.
        Here are the available tools: 
        <tools>
        {signatures}
        </tools>
        If you do not require function call to answer user query, respond in string format.
        In case of a function call, do not include a preamble statement, only return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
        <tool_call>{{"arguments": <args-dict>, "name": <function-name>}}</tool_call>"""),
    ("user", "{query}"),
    # ("assistant", "<tool_call>{{'arguments': {{'location': 'New Delhi', 'unit': 'celsius'}}, 'name': 'fn_get_weather'}}</tool_call>"),
    # ("assistant","<tool_response>{{'content': 'Now the weather in New Delhi is 22 celsius', 'name': 'fn_get_weather'}}</tool_response>")
    ]
)

chain = prompt | llm.bind_tools(tools)
response = chain.invoke(
    {'signatures':f"{tools_signature}",
     "query":"What is the current weather in New Delhi in celsius",
    #  "query":"What is the magic function of 10",
    #  "query":"Tell me a joke above batman"
    }
)

response

AIMessage(content='<tool_call>{"arguments": {"location": "New Delhi", "unit": "celsius"}, "name": "get_current_weather"}</tool_call>', response_metadata={'finish_reason': 'stop'}, id='run-34b68570-b3ad-4f79-b061-522d1ad6f538-0')

'51b82461-6bc7-4048-abae-54080d7c2eb1'

In [405]:
tool_call_id = f"{uuid.uuid4()}"

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant. You also have access to additional functions (tools) to assist with user queries. Function signatures are provided within <tools></tools> XML tags.
        You may call one or more functions to assist with the user query if you are unsure about the answer. Don't make assumptions about what values to plug into functions.
        When you make a tool call, wait for the tool to respond to generate the final answer. Tool response would be provided in <tool_response></tool_response> XML tags.
        Here are the available tools: 
        <tools>{signatures}</tools>
        If you do not require function call to answer user query, respond in string format.
        In case of a function call, do not include a preamble statement, only return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
        <tool_call>{{"arguments": <args-dict>, "name": <function-name>}}</tool_call>
    """),
    ("user", "{query}"),
    AIMessage(content = "<tool_call>{{'arguments': {{'location': 'New Delhi', 'unit': 'celsius'}}, 'name': 'fn_get_weather'}}</tool_call>"),
    ToolMessage(content = "<tool_response>{{'content': 'Now the weather in New Delhi is 22 celsius', 'name': 'fn_get_weather'}}</tool_response>", tool_call_id = tool_call_id)
    ]
)

chain = prompt #| llm.bind_tools(tools)
response = chain.invoke(
    {'signatures':f"{tools_signature}",
     "query":"What is the current weather in New Delhi in celsius",
    #  "query":"What is the magic function of 10",
    #  "query":"Tell me a joke above batman"
    }
)

response

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant. You also have access to additional functions (tools) to assist with user queries. Function signatures are provided within <tools></tools> XML tags.\n        You may call one or more functions to assist with the user query if you are unsure about the answer. Don\'t make assumptions about what values to plug into functions.\n        When you make a tool call, wait for the tool to respond to generate the final answer. Tool response would be provided in <tool_response></tool_response> XML tags.\n        Here are the available tools: \n        <tools>[{\'name\': \'get_current_weather\', \'description\': \'Get the current weather in a given location\', \'parameters\': {\'title\': \'WeatherInput\', \'type\': \'object\', \'properties\': {\'location\': {\'title\': \'Location\', \'description\': \'The city and state, e.g. San Francisco, CA\', \'type\': \'string\'}, \'unit\': {\'title\': \'Unit\', \'enum\': [\'celsius\'

# Misc Testing

In [ ]:
def fn_get_weather(location: str, unit: str):
    """Get the current weather in a given location"""
    return f"Now the weather in {location} is 22 {unit}"

fn_get_weather.__annotations__.items()

dict_items([('location', <class 'str'>), ('unit', <class 'str'>)])

In [7]:
few_shot = [
    {
        "example": "```\nSYSTEM: You are a helpful assistant who has access to functions. Use them if required\n<tools>[\n {\n \"name\": \"calculate_distance\",\n \"description\": \"Calculate the distance between two locations\",\n \"parameters\": {\n \"type\": \"object\",\n \"properties\": {\n \"origin\": {\n \"type\": \"string\",\n \"description\": \"The starting location\"\n },\n \"destination\": {\n \"type\": \"string\",\n \"description\": \"The destination location\"\n },\n \"mode\": {\n \"type\": \"string\",\n \"description\": \"The mode of transportation\"\n }\n },\n \"required\": [\n \"origin\",\n \"destination\",\n \"mode\"\n ]\n }\n },\n {\n \"name\": \"generate_password\",\n \"description\": \"Generate a random password\",\n \"parameters\": {\n \"type\": \"object\",\n \"properties\": {\n \"length\": {\n \"type\": \"integer\",\n \"description\": \"The length of the password\"\n }\n },\n \"required\": [\n \"length\"\n ]\n }\n }\n]\n\n</tools>\nUSER: Hi, I need to know the distance from New York to Los Angeles by car.\nASSISTANT:\n<tool_call>\n{\"arguments\": {\"origin\": \"New York\",\n \"destination\": \"Los Angeles\", \"mode\": \"car\"}, \"name\": \"calculate_distance\"}\n</tool_call>\n```\n"
    },
    {
        "example": "```\nSYSTEM: You are a helpful assistant with access to functions. Use them if required\n<tools>[\n {\n \"name\": \"calculate_distance\",\n \"description\": \"Calculate the distance between two locations\",\n \"parameters\": {\n \"type\": \"object\",\n \"properties\": {\n \"origin\": {\n \"type\": \"string\",\n \"description\": \"The starting location\"\n },\n \"destination\": {\n \"type\": \"string\",\n \"description\": \"The destination location\"\n },\n \"mode\": {\n \"type\": \"string\",\n \"description\": \"The mode of transportation\"\n }\n },\n \"required\": [\n \"origin\",\n \"destination\",\n \"mode\"\n ]\n }\n },\n {\n \"name\": \"generate_password\",\n \"description\": \"Generate a random password\",\n \"parameters\": {\n \"type\": \"object\",\n \"properties\": {\n \"length\": {\n \"type\": \"integer\",\n \"description\": \"The length of the password\"\n }\n },\n \"required\": [\n \"length\"\n ]\n }\n }\n]\n\n</tools>\nUSER: Can you help me generate a random password with a length of 8 characters?\nASSISTANT:\n<tool_call>\n{\"arguments\": {\"length\": 8}, \"name\": \"generate_password\"}\n</tool_call>\n```"
    }
]

In [8]:
print(few_shot[0]['example'])

```
SYSTEM: You are a helpful assistant who has access to functions. Use them if required
<tools>[
 {
 "name": "calculate_distance",
 "description": "Calculate the distance between two locations",
 "parameters": {
 "type": "object",
 "properties": {
 "origin": {
 "type": "string",
 "description": "The starting location"
 },
 "destination": {
 "type": "string",
 "description": "The destination location"
 },
 "mode": {
 "type": "string",
 "description": "The mode of transportation"
 }
 },
 "required": [
 "origin",
 "destination",
 "mode"
 ]
 }
 },
 {
 "name": "generate_password",
 "description": "Generate a random password",
 "parameters": {
 "type": "object",
 "properties": {
 "length": {
 "type": "integer",
 "description": "The length of the password"
 }
 },
 "required": [
 "length"
 ]
 }
 }
]

</tools>
USER: Hi, I need to know the distance from New York to Los Angeles by car.
ASSISTANT:
<tool_call>
{"arguments": {"origin": "New York",
 "destination": "Los Angeles", "mode": "car"}, "na

In [36]:
import json

import xml.etree.ElementTree as ET 


xmlstring = """
<tools>[
 {
 "name": "calculate_distance",
 "description": "Calculate the distance between two locations",
 "parameters": {
 "type": "object",
 "properties": {
 "origin": {
 "type": "string",
 "description": "The starting location"
 },
 "destination": {
 "type": "string",
 "description": "The destination location"
 },
 "mode": {
 "type": "string",
 "description": "The mode of transportation"
 }
 },
 "required": [
 "origin",
 "destination",
 "mode"
 ]
 }
 },
 {
 "name": "generate_password",
 "description": "Generate a random password",
 "parameters": {
 "type": "object",
 "properties": {
 "length": {
 "type": "integer",
 "description": "The length of the password"
 }
 },
 "required": [
 "length"
 ]
 }
 }
]

</tools>
"""

# xmlstring = few_shot[0]['example']

# parser = ET.XMLParser(encoding="utf-8")
# output = ET.fromstring(xmlstring, parser=parser)

output = ET.fromstring(xmlstring)


print(output)

<Element 'tools' at 0x110603330>


In [37]:
print(json.dumps(json.loads(output.text), indent = 4))

[
    {
        "name": "calculate_distance",
        "description": "Calculate the distance between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "origin": {
                    "type": "string",
                    "description": "The starting location"
                },
                "destination": {
                    "type": "string",
                    "description": "The destination location"
                },
                "mode": {
                    "type": "string",
                    "description": "The mode of transportation"
                }
            },
            "required": [
                "origin",
                "destination",
                "mode"
            ]
        }
    },
    {
        "name": "generate_password",
        "description": "Generate a random password",
        "parameters": {
            "type": "object",
            "properties": {
                "length": {
 

In [46]:
from langchain_core.pydantic_v1 import BaseModel
from typing import Optional, Dict

class FunctionDefinition(BaseModel):
    name: str
    description: Optional[str] = None
    parameters: Optional[Dict[str, object]] = None

print(json.dumps(json.loads(FunctionDefinition.schema_json()), indent=4))

{
    "title": "FunctionDefinition",
    "type": "object",
    "properties": {
        "name": {
            "title": "Name",
            "type": "string"
        },
        "description": {
            "title": "Description",
            "type": "string"
        },
        "parameters": {
            "title": "Parameters",
            "type": "object"
        }
    },
    "required": [
        "name"
    ]
}


In [48]:
defintn = FunctionDefinition(
            name='sample_fn_name',
            description='sample fn description',
        )

defintn.dict()

{'name': 'sample_fn_name',
 'description': 'sample fn description',
 'parameters': None}